In [4]:
# Function to safely recall a stored variable
def recall_stored_variable(var_name, default_value):
    try:
        get_ipython().run_line_magic('store', f'-r {var_name}')
        return globals()[var_name]
    except KeyError:
        return default_value

In [8]:
# Ensure you retrieve the bucket_name and other needed variables from previous notebooks
%store -r bucket_name

import boto3
import sagemaker
from sagemaker.model_monitor import DefaultModelMonitor, DatasetFormat
from sagemaker.model import Model
from sagemaker.session import Session
from botocore.exceptions import ClientError

# Initialize SageMaker session and role
session = Session()
role = sagemaker.get_execution_role()
'''
# Set up model monitoring variables
monitoring_schedule_name = "housing-model-monitoring-schedule"
baseline_dataset = f"s3://{bucket_name}/processed/housing/train.csv"

# Model artifact location and endpoint name
model_data = 's3://housing-dataset-5435xx/housing/model/xgboost-2024-06-20-02-20-59-393/output/model.tar.gz'
endpoint_name = "housing-endpoint"
image_uri = "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest"
'''

baseline_dataset = recall_stored_variable('baseline_dataset', f"s3://{bucket_name}/processed/housing/train.csv")
model_data = recall_stored_variable('model_data', 's3://housing-dataset-5435xx/housing/model/xgboost-2024-06-20-02-20-59-393/output/model.tar.gz')
image_uri = recall_stored_variable('image_uri', "811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest")
endpoint_name = "housing-endpoint"
monitoring_schedule_name = "housing-model-monitoring-schedule"

# Your existing code using the variables
print(f"Bucket Name: {bucket_name}")
print(f"Baseline Dataset: {baseline_dataset}")
print(f"Model Data: {model_data}")
print(f"Image URI: {image_uri}")
print(f"Endpoint Name: {endpoint_name}")



# Check if endpoint exists
sm_client = boto3.client('sagemaker')

try:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_exists = response['EndpointStatus'] in ['InService', 'Creating', 'Updating']
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        endpoint_exists = False
    else:
        raise e

# Deploy the model to an endpoint if it does not exist
if not endpoint_exists:
    print(f"Endpoint '{endpoint_name}' does not exist. Deploying the model...")
    model = Model(model_data=model_data, role=role, image_uri=image_uri)
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name,
        wait=True,
    )
else:
    print(f"Endpoint '{endpoint_name}' already exists. Skipping deployment...")

# Initialize the DefaultModelMonitor
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session
)

# Suggest baseline from the training data
monitor.suggest_baseline(
    baseline_dataset=baseline_dataset,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=f"s3://{bucket_name}/baseline_output",
)

# Create monitoring schedule
monitor.create_monitoring_schedule(
    monitor_schedule_name=monitoring_schedule_name,
    endpoint_input=endpoint_name,
    output_s3_uri=f"s3://{bucket_name}/monitoring_output",
    statistics=monitor.baseline_statistics(),
    constraints=monitor.suggested_constraints(),
    schedule_cron_expression='cron(0 * ? * * *)'
)

# Describe the monitoring schedule to verify setup
monitor.describe_schedule()

# Retrieve the monitoring schedule status
monitoring_schedule_status = monitor.describe_schedule()
print(monitoring_schedule_status)


no stored variable or alias baseline_dataset
no stored variable or alias model_data
no stored variable or alias image_uri
Bucket Name: housing-dataset-5435xx
Baseline Dataset: s3://housing-dataset-5435xx/processed/housing/train.csv
Model Data: s3://housing-dataset-5435xx/housing/model/xgboost-2024-06-20-02-20-59-393/output/model.tar.gz
Image URI: 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest
Endpoint Name: housing-endpoint


SyntaxError: 'break' outside loop (3314107271.py, line 37)

In [6]:
# Delete the endpoint
try:
    predictor.delete_endpoint()
    print(f"Endpoint '{endpoint_name}' deleted successfully.")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print(f"Endpoint '{endpoint_name}' does not exist.")
    else:
        raise e

# Delete the endpoint configuration
try:
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Endpoint configuration '{endpoint_name}' deleted successfully.")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print(f"Endpoint configuration '{endpoint_name}' does not exist.")
    else:
        raise e

# Delete the monitoring schedule
try:
    sm_client.delete_monitoring_schedule(MonitoringScheduleName="housing-model-monitoring-schedule")
    print("Monitoring schedule deleted successfully.")
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFound':
        print("Monitoring schedule does not exist.")
    else:
        raise e

NameError: name 'predictor' is not defined